In [ ]:
# -*- coding: utf-8 -*-
"""Recomendação PI6DSM - GAMES.ipynb

Sistema de recomendação de games adaptado da base MovieLens
"""

# pip install scikit-surprise
!pip install numpy==1.26.4 scikit-surprise

# Importação de bibliotecas e configuração inicial
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
from surprise import Dataset, Reader, KNNBasic

Mounted at /content/drive


In [ ]:
# Carregamento da base de dados
print("📁 CARREGANDO BASE DE DADOS DE GAMES")
print("=" * 50)

# Carregar a base de games
games = pd.read_csv('/content/drive/My Drive/datasets/games_blt3.csv')
print(f"✅ Base carregada: {games.shape[0]} jogos, {games.shape[1]} atributos")
print(f"📊 Colunas disponíveis: {list(games.columns)}")

# Mostrar primeiras linhas
print("\n🔍 Primeiros 5 jogos da base:")
games.head()

📁 CARREGANDO BASE DE DADOS DE GAMES
✅ Base carregada: 2497 jogos, 11 atributos
📊 Colunas disponíveis: ['name', 'release_date', 'required_age', 'price', 'header_image', 'positive', 'negative', 'recommendations', 'genres', 'categories', 'description']

🔍 Primeiros 5 jogos da base:


,name,release_date,required_age,price,header_image,positive,negative,recommendations,genres,categories,description
0,Galactic Bowling,20081021.0,0.0,19.99,https://cdn.akamai.steamstatic.com/steam/apps/...,0.0,6.0,30.0,"Singleplayer,Multiplayer,Steam Achievements,Pa...","Indie,Casual,Sports,Bowling",d
1,Train Bandit,20171012.0,0.0,0.99,https://cdn.akamai.steamstatic.com/steam/apps/...,0.0,53.0,12.0,"Singleplayer,Steam Achievements,Full controlle...","Indie,Action,Pixel Graphics,2D,Retro,Arcade,Sc...",THE LAW Looks to be a showdown atop a train Th...
2,Jolt Project,20211117.0,0.0,4.99,https://cdn.akamai.steamstatic.com/steam/apps/...,0.0,0.0,0.0,Singleplayer,NaN,Jolt Project The army now has a new robotics p...
3,Henosis 12,20200723.0,0.0,5.99,NaN,NaN,NaN,NaN,NaN,NaN,HENOSIS 12 is a mysterious 2D Platform Puzzler...
4,Two Weeks in Painland,20200203.0,0.0,0.00,https://cdn.akamai.steamstatic.com/steam/apps/...,0.0,50.0,17.0,"Singleplayer,Steam Achievements","Indie,Adventure,Nudity,Violent,Sexual Content,...",ABOUT THE GAME Play as a hacker who has arrang...


In [ ]:
# Célula 3: Criação de ratings simulados
print("🎯 CRIANDO RATINGS SIMULADOS")
print("=" * 50)

def criar_ratings_simulados(games_df, num_usuarios=100):
    """Cria ratings simulados baseados nas avaliações positivas/negativas"""
    ratings_data = []

    for user_id in range(1, num_usuarios + 1):
        num_jogos_avaliados = np.random.randint(5, 20)
        jogos_avaliados = games_df.sample(num_jogos_avaliados)

        for _, jogo in jogos_avaliados.iterrows():
            positivas = jogo['positive']
            negativas = jogo['negative']
            total_avaliacoes = positivas + negativas

            if total_avaliacoes > 0:
                score_base = positivas / total_avaliacoes
                rating = max(1, min(5, round(score_base * 4 + 1 + np.random.normal(0, 0.5))))
            else:
                rating = np.random.randint(2, 5)

            ratings_data.append({
                'user_id': user_id,
                'item_id': jogo['name'],
                'rating': rating
            })

    return pd.DataFrame(ratings_data)

# Criar ratings simulados
ratings_simulados = criar_ratings_simulados(games)
print(f"✅ Ratings criados: {len(ratings_simulados)} avaliações de {ratings_simulados['user_id'].nunique()} usuários")
print(f"📈 Amostra dos ratings:")
ratings_simulados.head()

🎯 CRIANDO RATINGS SIMULADOS
✅ Ratings criados: 1247 avaliações de 100 usuários
📈 Amostra dos ratings:


,user_id,item_id,rating
0,1,LOGistICAL ABC Islands,1
1,1,VR JAPAN,1
2,1,Atlas Engine,2
3,1,Color by Number Monster Heroes,1
4,1,ANCIENT SOULS The Governor,4


In [ ]:
# Célula 4: Cálculo de métricas para análise
print("📊 CALCULANDO MÉTRICAS PARA ANÁLISE")
print("=" * 50)

# Calcular total de votos e notas médias
total_de_votos = ratings_simulados["item_id"].value_counts()
notas_medias = ratings_simulados.groupby("item_id").mean()["rating"]

# Adicionar ao dataframe principal
games['total_de_votos'] = games['name'].map(total_de_votos).fillna(0)
games['nota_media'] = games['name'].map(notas_medias).fillna(0)

print("✅ Métricas calculadas com sucesso")
print(f"📋 Jogos com avaliações: {(games['total_de_votos'] > 0).sum()}")
print(f"🎯 Amostra dos dados com métricas:")
games[['name', 'nota_media', 'total_de_votos', 'genres']].head()

📊 CALCULANDO MÉTRICAS PARA ANÁLISE
✅ Métricas calculadas com sucesso
📋 Jogos com avaliações: 979
🎯 Amostra dos dados com métricas:


,name,nota_media,total_de_votos,genres
0,Galactic Bowling,1.0,1.0,"Singleplayer,Multiplayer,Steam Achievements,Pa..."
1,Train Bandit,1.0,1.0,"Singleplayer,Steam Achievements,Full controlle..."
2,Jolt Project,0.0,0.0,Singleplayer
3,Henosis 12,3.0,2.0,NaN
4,Two Weeks in Painland,1.0,1.0,"Singleplayer,Steam Achievements"


In [ ]:
# Célula 5: Análise exploratória - Jogos mais populares (CORRIGIDA)
print("🏆 ANÁLISE EXPLORATÓRIA - JOGOS MAIS POPULARES")
print("=" * 50)

print("🎯 TOP 10 JOGOS MAIS VOTADOS:")
jogos_populares = games.sort_values("total_de_votos", ascending=False).head(10)

for i, (_, jogo) in enumerate(jogos_populares.iterrows(), 1):
    nome = jogo['name'][:50] if len(jogo['name']) > 50 else jogo['name']
    votos = int(jogo['total_de_votos'])  # Converter para inteiro
    nota = jogo['nota_media']
    print(f"{i:2d}. {nome:50} | Votos: {votos:3d} | Nota: {nota:.2f}")

🏆 ANÁLISE EXPLORATÓRIA - JOGOS MAIS POPULARES
🎯 TOP 10 JOGOS MAIS VOTADOS:
 1. Switchball HD Puzzle Platformer                    | Votos:   4 | Nota: 3.75
 2. Gunkid 99                                          | Votos:   4 | Nota: 1.00
 3. Demon Speakeasy                                    | Votos:   4 | Nota: 3.25
 4. Mystery Case Files Crossfade Collectors Edition    | Votos:   4 | Nota: 3.25
 5. Helltower                                          | Votos:   4 | Nota: 1.00
 6. Gray Zone Playtest                                 | Votos:   4 | Nota: 3.25
 7. Elder Chaos                                        | Votos:   3 | Nota: 1.00
 8. Razenroth                                          | Votos:   3 | Nota: 1.33
 9. Alien Planet                                       | Votos:   3 | Nota: 1.33
10. Navy Field 2 Conqueror of the Ocean                | Votos:   3 | Nota: 2.33


In [ ]:
# Célula 6: Análise exploratória - Jogos melhor avaliados (CORRIGIDA)
print("⭐ ANÁLISE EXPLORATÓRIA - JOGOS MELHOR AVALIADOS")
print("=" * 50)

print("🏅 TOP 10 JOGOS MELHOR AVALIADOS (mínimo 5 votos):")
jogos_bem_avaliados = games[games['total_de_votos'] >= 5].sort_values("nota_media", ascending=False).head(10)

for i, (_, jogo) in enumerate(jogos_bem_avaliados.iterrows(), 1):
    nome = jogo['name'][:50] if len(jogo['name']) > 50 else jogo['name']
    votos = int(jogo['total_de_votos'])  # Converter para inteiro
    nota = jogo['nota_media']
    print(f"{i:2d}. {nome:50} | Nota: {nota:.2f} | Votos: {votos:3d}")

⭐ ANÁLISE EXPLORATÓRIA - JOGOS MELHOR AVALIADOS
🏅 TOP 10 JOGOS MELHOR AVALIADOS (mínimo 5 votos):


In [ ]:
# Célula 7: Configuração e treinamento do modelo K-NN
print("🤖 CONFIGURANDO E TREINANDO MODELO K-NN")
print("=" * 50)

# Configurar reader e dados para o Surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings_simulados[['user_id', 'item_id', 'rating']], reader)
trainset = data.build_full_trainset()

# Configurar e treinar modelo
sim_options = {
    'name': 'cosine',
    'user_based': False  # Baseado em similaridade entre itens (jogos)
}
model = KNNBasic(sim_options=sim_options)
model.fit(trainset)

print("✅ Modelo K-NN treinado com sucesso!")
print(f"   👥 Usuários no sistema: {trainset.n_users}")
print(f"   🎮 Jogos no sistema: {trainset.n_items}")
print(f"   ⭐ Ratings utilizados: {trainset.n_ratings}")

🤖 CONFIGURANDO E TREINANDO MODELO K-NN
Computing the cosine similarity matrix...
Done computing similarity matrix.
✅ Modelo K-NN treinado com sucesso!
   👥 Usuários no sistema: 100
   🎮 Jogos no sistema: 979
   ⭐ Ratings utilizados: 1247


In [ ]:
# Célula 8: Função de recomendação por similaridade
print("🎮 FUNÇÃO DE RECOMENDAÇÃO POR SIMILARIDADE")
print("=" * 50)

def recomendar_jogos_similares(nome_jogo, modelo, trainset, games_df, k=5):
    """Recomenda jogos similares usando algoritmo K-NN"""
    try:
        # Converter nome para ID interno do Surprise
        jogo_inner_id = trainset.to_inner_iid(nome_jogo)

        # Encontrar vizinhos mais próximos
        neighbors = modelo.get_neighbors(jogo_inner_id, k=k)

        # Converter IDs internos de volta para nomes
        jogos_recomendados = [trainset.to_raw_iid(inner_id) for inner_id in neighbors]

        print(f"🎯 Jogos similares a '{nome_jogo}':")
        print("=" * 70)

        # Mostrar informações dos jogos recomendados
        for i, jogo_nome in enumerate(jogos_recomendados, 1):
            jogo_info = games_df[games_df['name'] == jogo_nome].iloc[0]
            generos = jogo_info.get('genres', 'N/A')
            preco = jogo_info.get('price', 'N/A')
            nota = jogo_info.get('nota_media', 'N/A')

            print(f"{i}. {jogo_nome}")
            print(f"   🏷️  Gêneros: {generos}")
            print(f"   💰 Preço: ${preco}")
            if nota != "N/A":
                print(f"   ⭐ Nota: {nota:.2f}")
            print()

        return jogos_recomendados

    except ValueError:
        print(f"❌ Jogo '{nome_jogo}' não encontrado no sistema de recomendação.")
        disponiveis = list(trainset._raw2inner_id_items.keys())[:8]
        print("💡 Jogos disponíveis para recomendação:")
        for jogo in disponiveis:
            print(f"   - {jogo}")
        return []

print("✅ Função 'recomendar_jogos_similares' criada com sucesso!")

🎮 FUNÇÃO DE RECOMENDAÇÃO POR SIMILARIDADE
✅ Função 'recomendar_jogos_similares' criada com sucesso!


In [ ]:
# Célula 9: Função de recomendação por gênero
print("🎯 FUNÇÃO DE RECOMENDAÇÃO POR GÊNERO")
print("=" * 50)

def recomendar_por_genero(genero, games_df, top_n=5):
    """Recomenda jogos por gênero baseado na nota média"""
    # Filtrar jogos do gênero especificado
    jogos_filtrados = games_df[games_df['genres'].str.contains(genero, na=False)]

    if jogos_filtrados.empty:
        print(f"❌ Nenhum jogo encontrado para o gênero '{genero}'")
        return []

    # Ordenar por nota média e pegar os melhores
    recomendados = jogos_filtrados.sort_values('nota_media', ascending=False).head(top_n)

    print(f"🎮 Top {top_n} jogos do gênero '{genero}':")
    print("=" * 60)

    # Mostrar resultados
    for i, (_, jogo) in enumerate(recomendados.iterrows(), 1):
        print(f"{i}. {jogo['name']}")
        print(f"   ⭐ Nota: {jogo['nota_media']:.2f}")
        print(f"   💰 Preço: ${jogo['price']}")
        print(f"   👍 Avaliações positivas: {jogo['positive']}")
        print()

    return recomendados['name'].tolist()

print("✅ Função 'recomendar_por_genero' criada com sucesso!")

🎯 FUNÇÃO DE RECOMENDAÇÃO POR GÊNERO
✅ Função 'recomendar_por_genero' criada com sucesso!


In [ ]:
# Célula 10: Lista de jogos disponíveis para recomendação
print("📋 JOGOS DISPONÍVEIS PARA RECOMENDAÇÃO")
print("=" * 50)

jogos_disponiveis = list(trainset._raw2inner_id_items.keys())
print(f"Total de jogos no sistema: {len(jogos_disponiveis)}")
print("\nPrimeiros 15 jogos disponíveis:")
for i, jogo in enumerate(jogos_disponiveis[:15], 1):
    print(f"{i:2d}. {jogo}")

📋 JOGOS DISPONÍVEIS PARA RECOMENDAÇÃO
Total de jogos no sistema: 979

Primeiros 15 jogos disponíveis:
 1. LOGistICAL ABC Islands
 2. VR JAPAN
 3. Atlas Engine
 4. Color by Number Monster Heroes
 5. ANCIENT SOULS The Governor
 6. Tiki Tiki The Tropical Memory Game
 7. The Architects Game
 8. Super Plexis
 9. NOBUNAGAS AMBITION Soutenroku with Power Up Kit
10. Galactic Bowling
11.     s 12 s   
12. Shank
13. Idol Manager
14. Squirgle
15. Dead Space


In [ ]:
# Célula 11 CORRIGIDA: Demonstração - Recomendação por similaridade
print("🔍 DEMONSTRAÇÃO: RECOMENDAÇÃO POR SIMILARIDADE")
print("=" * 50)

if jogos_disponiveis:
    # Em vez de sempre pegar o primeiro, vamos variar
    import random
    jogos_para_testar = random.sample(jogos_disponiveis, min(3, len(jogos_disponiveis)))

    for jogo_teste in jogos_para_testar:
        print(f"\n🧪 Testando com o jogo: '{jogo_teste}'")
        print("-" * 50)

        recomendacoes = recomendar_jogos_similares(jogo_teste, model, trainset, games)

        if recomendacoes:
            print(f"✅ {len(recomendacoes)} recomendações geradas!")
        else:
            print("❌ Nenhuma recomendação gerada.")
else:
    print("❌ Nenhum jogo disponível para teste.")

🔍 DEMONSTRAÇÃO: RECOMENDAÇÃO POR SIMILARIDADE

🧪 Testando com o jogo: 'The Occupation'
--------------------------------------------------
🎯 Jogos similares a 'The Occupation':
1. Project Lounge
   🏷️  Gêneros: Singleplayer,Multiplayer,Steam Achievements,Steam Trading Cards,Steam Workshop,Steam Cloud,Includes level editor
   💰 Preço: $0.99
   ⭐ Nota: 1.00

2. Dialogue Box The Road Less Traveled
   🏷️  Gêneros: Singleplayer,Steam Achievements
   💰 Preço: $0.0
   ⭐ Nota: 2.33

3. Drone VR
   🏷️  Gêneros: nan
   💰 Preço: $0.89
   ⭐ Nota: 3.50

4. Teleporter World of Gamers Alpha
   🏷️  Gêneros: nan
   💰 Preço: $0.0
   ⭐ Nota: 3.00

5. Woodsalt
   🏷️  Gêneros: Singleplayer,Full controller support
   💰 Preço: $24.99
   ⭐ Nota: 2.00

✅ 5 recomendações geradas!

🧪 Testando com o jogo: 'Tiny Wheels'
--------------------------------------------------
🎯 Jogos similares a 'Tiny Wheels':
1. Worm Runner
   🏷️  Gêneros: Singleplayer,Steam Achievements
   💰 Preço: $0.99
   ⭐ Nota: 1.00

2. Cultivation

In [ ]:
# Célula 12: Demonstração - Recomendação por gênero
print("🔍 DEMONSTRAÇÃO: RECOMENDAÇÃO POR GÊNERO")
print("=" * 50)

# Encontrar gêneros disponíveis para teste
generos_disponiveis = games['genres'].dropna()
if not generos_disponiveis.empty:
    # Pegar o primeiro gênero da lista
    primeiro_genero = generos_disponiveis.iloc[0]
    if ',' in primeiro_genero:
        primeiro_genero = primeiro_genero.split(',')[0].strip()

    print(f"Testando com o gênero: '{primeiro_genero}'")
    print("-" * 50)

    recomendacoes_genero = recomendar_por_genero(primeiro_genero, games)

    if recomendacoes_genero:
        print(f"✅ {len(recomendacoes_genero)} recomendações por gênero geradas!")
    else:
        print("❌ Nenhuma recomendação por gênero gerada.")
else:
    print("❌ Nenhum gênero disponível para teste.")

🔍 DEMONSTRAÇÃO: RECOMENDAÇÃO POR GÊNERO
Testando com o gênero: 'Singleplayer'
--------------------------------------------------
🎮 Top 5 jogos do gênero 'Singleplayer':
1. Memoir
   ⭐ Nota: 4.00
   💰 Preço: $0.0
   👍 Avaliações positivas: 0.0

2. She Salutes
   ⭐ Nota: 4.00
   💰 Preço: $7.99
   👍 Avaliações positivas: 0.0

3. Peblito Rock and Roll
   ⭐ Nota: 4.00
   💰 Preço: $0.0
   👍 Avaliações positivas: 0.0

4. The Fun of Asmodius
   ⭐ Nota: 4.00
   💰 Preço: $10.99
   👍 Avaliações positivas: 0.0

5. Key
   ⭐ Nota: 4.00
   💰 Preço: $0.99
   👍 Avaliações positivas: 0.0

✅ 5 recomendações por gênero geradas!


In [ ]:
# Célula 13: Sistema de recomendação interativo
print("🎮 SISTEMA DE RECOMENDAÇÃO INTERATIVO")
print("=" * 50)

# No sistema interativo, adicione esta opção:
def sistema_recomendacao_interativo_melhorado():
    """Sistema interativo melhorado"""
    print("\n" + "="*60)
    print("🎮 SISTEMA DE RECOMENDAÇÃO DE GAMES - MODO INTERATIVO")
    print("="*60)

    while True:
        print("\n📝 OPÇÕES DISPONÍVEIS:")
        print("1. 🎯 Recomendar jogos similares")
        print("2. 🏷️  Recomendar por gênero")
        print("3. 📋 Listar jogos disponíveis")
        print("4. 🎲 Recomendação aleatória (demo)")
        print("5. ❌ Sair do sistema")

        opcao = input("\nEscolha uma opção (1-5): ").strip()

        if opcao == '1':
            print("\n" + "-"*40)
            jogo = input("Digite o nome exato do jogo: ").strip()
            if jogo:
                recomendar_jogos_similares(jogo, model, trainset, games)
            else:
                print("❌ Nome do jogo não pode estar vazio.")

        elif opcao == '2':
            print("\n" + "-"*40)
            genero = input("Digite o gênero: ").strip()
            if genero:
                recomendar_por_genero(genero, games)
            else:
                print("❌ Gênero não pode estar vazio.")

        elif opcao == '3':
            print("\n📋 JOGOS DISPONÍVEIS (primeiros 20):")
            print("-" * 40)
            for i, jogo in enumerate(jogos_disponiveis[:20], 1):
                print(f"{i:2d}. {jogo}")

        elif opcao == '4':
            print("\n🎲 RECOMENDAÇÃO ALEATÓRIA:")
            print("-" * 40)
            if jogos_disponiveis:
                jogo_aleatorio = random.choice(jogos_disponiveis)
                print(f"Jogo selecionado aleatoriamente: '{jogo_aleatorio}'")
                recomendar_jogos_similares(jogo_aleatorio, model, trainset, games)
            else:
                print("❌ Nenhum jogo disponível.")

        elif opcao == '5':
            print("\n👋 Saindo do sistema... Obrigado por usar!")
            break

        else:
            print("❌ Opção inválida. Por favor, escolha 1, 2, 3, 4 ou 5.")

# Substitua a chamada original por:
#sistema_recomendacao_interativo_melhorado()

🎮 SISTEMA DE RECOMENDAÇÃO INTERATIVO


In [ ]:
# Célula 14: Conclusão e resumo final
print("✅ SISTEMA DE RECOMENDAÇÃO CONCLUÍDO!")
print("=" * 50)

print("\n🎯 RESUMO DO SISTEMA IMPLEMENTADO:")
print(f"   • 📁 Base de dados: {games.shape[0]} jogos carregados")
print(f"   • ⭐ Ratings simulados: {len(ratings_simulados)} avaliações")
print(f"   • 🤖 Algoritmo: K-NN com similaridade por cosseno")
print(f"   • 🎮 Jogos no sistema: {trainset.n_items}")
print(f"   • 👥 Usuários simulados: {trainset.n_users}")

print("\n🔧 FUNCIONALIDADES DISPONÍVEIS:")
print("   1. Recomendação por similaridade (K-NN)")
print("   2. Recomendação por gênero")
print("   3. Análise de jogos populares")
print("   4. Sistema interativo para testes")

print("\n🚀 PRÓXIMOS PASSOS:")
print("   • Execute 'sistema_recomendacao_interativo()' para testar")
print("   • Experimente com diferentes jogos e gêneros")
print("   • Analise as recomendações geradas")

print("\n" + "=" * 50)
print("🎮 SISTEMA PRONTO PARA USO!")
print("=" * 50)

✅ SISTEMA DE RECOMENDAÇÃO CONCLUÍDO!

🎯 RESUMO DO SISTEMA IMPLEMENTADO:
   • 📁 Base de dados: 2497 jogos carregados
   • ⭐ Ratings simulados: 1247 avaliações
   • 🤖 Algoritmo: K-NN com similaridade por cosseno
   • 🎮 Jogos no sistema: 979
   • 👥 Usuários simulados: 100

🔧 FUNCIONALIDADES DISPONÍVEIS:
   1. Recomendação por similaridade (K-NN)
   2. Recomendação por gênero
   3. Análise de jogos populares
   4. Sistema interativo para testes

🚀 PRÓXIMOS PASSOS:
   • Execute 'sistema_recomendacao_interativo()' para testar
   • Experimente com diferentes jogos e gêneros
   • Analise as recomendações geradas

🎮 SISTEMA PRONTO PARA USO!


In [ ]:
# Célula extra: Análise da qualidade das recomendações
print("🔍 ANALISANDO QUALIDADE DAS RECOMENDAÇÕES")
print("=" * 50)

def analisar_recomendacoes(modelo, trainset, games_df, num_testes=5):
    """Analisa recomendações para vários jogos diferentes"""
    jogos_amostra = random.sample(jogos_disponiveis, min(num_testes, len(jogos_disponiveis)))

    print(f"📊 Analisando {len(jogos_amostra)} recomendações diferentes:")
    print("=" * 60)

    for i, jogo in enumerate(jogos_amostra, 1):
        print(f"\n{i}. 🎮 Jogo base: '{jogo}'")
        try:
            jogo_inner_id = trainset.to_inner_iid(jogo)
            neighbors = modelo.get_neighbors(jogo_inner_id, k=3)

            jogos_recomendados = [trainset.to_raw_iid(inner_id) for inner_id in neighbors]

            print(f"   📋 Recomendações: {jogos_recomendados}")

            # Verificar similaridade de gêneros
            jogo_base_info = games_df[games_df['name'] == jogo].iloc[0]
            generos_base = jogo_base_info.get('genres', 'N/A')
            print(f"   🏷️  Gêneros base: {generos_base}")

            for rec in jogos_recomendados:
                rec_info = games_df[games_df['name'] == rec].iloc[0]
                generos_rec = rec_info.get('genres', 'N/A')
                print(f"   → {rec} | Gêneros: {generos_rec}")

        except ValueError:
            print(f"   ❌ Jogo não disponível para recomendação")

    print(f"\n✅ Análise concluída! Verifique se as recomendações fazem sentido.")

# Executar análise
analisar_recomendacoes(model, trainset, games)

🔍 ANALISANDO QUALIDADE DAS RECOMENDAÇÕES
📊 Analisando 5 recomendações diferentes:

1. 🎮 Jogo base: 'Peblito Rock and Roll'
   📋 Recomendações: ['StarPit Starship', 'Eternal Dread', 'Chowderchu']
   🏷️  Gêneros base: Singleplayer
   → StarPit Starship | Gêneros: Singleplayer,Steam Achievements,Full controller support,Steam Cloud,Steam Leaderboards
   → Eternal Dread | Gêneros: Singleplayer,Steam Achievements,Steam Cloud
   → Chowderchu | Gêneros: Singleplayer,Steam Achievements,Steam Trading Cards

2. 🎮 Jogo base: 'SteamCity Chronicles Rise Of The Rose'
   📋 Recomendações: ['Arcsaber VR', 'Goofy Golf Remastered Steam Edition', 'Adventure Farm VR']
   🏷️  Gêneros base: Singleplayer
   → Arcsaber VR | Gêneros: Singleplayer,Steam Cloud
   → Goofy Golf Remastered Steam Edition | Gêneros: Singleplayer
   → Adventure Farm VR | Gêneros: Singleplayer

3. 🎮 Jogo base: 'The Deer'
   📋 Recomendações: ['Terror In The Atomic Desert', 'QUBE Directors Cut', 'Hero of the Demon']
   🏷️  Gêneros base: Si

In [ ]:
sistema_recomendacao_interativo_melhorado()


🎮 SISTEMA DE RECOMENDAÇÃO DE GAMES - MODO INTERATIVO

📝 OPÇÕES DISPONÍVEIS:
1. 🎯 Recomendar jogos similares
2. 🏷️  Recomendar por gênero
3. 📋 Listar jogos disponíveis
4. 🎲 Recomendação aleatória (demo)
5. ❌ Sair do sistema

🎲 RECOMENDAÇÃO ALEATÓRIA:
----------------------------------------
Jogo selecionado aleatoriamente: 'Squares'
🎯 Jogos similares a 'Squares':
1. Ping Pong League
   🏷️  Gêneros: Singleplayer,Multiplayer
   💰 Preço: $0.0
   ⭐ Nota: 1.00

2. Worm Runner
   🏷️  Gêneros: Singleplayer,Steam Achievements
   💰 Preço: $0.99
   ⭐ Nota: 1.00

3. Cultivation Tales
   🏷️  Gêneros: Singleplayer,Multiplayer,PvP,Online PvP,Coop,Online Coop,InApp Purchases,Partial Controller Support
   💰 Preço: $19.99
   ⭐ Nota: 1.50

4. RoadPunk
   🏷️  Gêneros: Singleplayer,Steam Achievements,Full controller support,Steam Cloud,Stats,Steam Leaderboards
   💰 Preço: $0.99
   ⭐ Nota: 1.00

5. FLYING PUDDING
   🏷️  Gêneros: Singleplayer
   💰 Preço: $3.99
   ⭐ Nota: 1.00


📝 OPÇÕES DISPONÍVEIS:
1. 🎯 Re